# LPBF Optimizer Workflow Example

This notebook demonstrates the complete workflow of the physics-informed, AI-driven optimizer for Laser Powder Bed Fusion (LPBF) manufacturing processes. We'll walk through each step from FEA simulation to optimization.

In [ ]:
# Import necessary libraries
import os
import sys
import numpy as np
import torch
import h5py
import matplotlib.pyplot as plt
from pathlib import Path
import yaml

# Add parent directory to path to import local modules
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import project modules
from src.pinn.model import PINN
from src.pinn.physics import compute_physics_loss
from src.pinn.train import PINNTrainer
from src.optimiser.nsga3 import NSGAOptimizer
from src.optimiser.bayesopt import BayesianOptimizer

# Set up plotting
plt.style.use('seaborn-whitegrid')
%matplotlib inline

## 1. Configuration

First, let's load our configuration from the params.yaml file.

In [ ]:
# Load configuration
config_path = Path('../data/params.yaml')

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# Show material properties
print("Material Properties (Ti-6Al-4V):")
for key, value in config['material_properties'].items():
    print(f"  {key}: {value}")

## 2. Data Exploration

Next, let's explore the FEA simulation data. In a real scenario, this would be loaded from FEA simulation results. For this demo, we'll create some synthetic data.

In [ ]:
# Create some synthetic FEA data for demonstration
def generate_synthetic_data(n_samples=1000):
    """
    Generate synthetic data that mimics FEA simulation results
    """
    # Process parameters
    P = np.random.uniform(150, 400, n_samples)  # Laser power (W)
    v = np.random.uniform(500, 1500, n_samples)  # Scan speed (mm/s)
    h = np.random.uniform(0.05, 0.15, n_samples)  # Hatch spacing (mm)
    theta = np.random.uniform(0, 90, n_samples)  # Scan angle (degrees)
    l_island = np.random.uniform(2, 10, n_samples)  # Island size (mm)
    layer_thickness = np.random.uniform(0.02, 0.06, n_samples)  # Layer thickness (mm)
    
    # Spatial coordinates
    x = np.random.uniform(-5, 5, n_samples)
    y = np.random.uniform(-5, 5, n_samples)
    z = np.random.uniform(0, 1, n_samples)
    t = np.random.uniform(0, 1, n_samples)  # Time
    
    # Create input array: [P, v, h, theta, l_island, layer_thickness, x, y, z, t]
    inputs = np.column_stack([P, v, h, theta, l_island, layer_thickness, x, y, z, t])
    
    # Generate synthetic outputs
    # Residual stress (MPa) - increases with P, decreases with v
    residual_stress = 200 + 0.5*P - 0.1*v + 50*np.random.randn(n_samples)
    
    # Porosity (%) - decreases with Energy Density (P/(v*h))
    energy_density = P / (v * h)
    porosity = 5 - 0.1*np.log(energy_density) + 1*np.random.randn(n_samples)
    porosity = np.clip(porosity, 0.1, 10) / 100  # Convert to fraction and clip to reasonable range
    
    # Geometric accuracy (dimensionless) - better with moderate energy density
    geo_accuracy = 0.9 + 0.1*np.exp(-(energy_density - 0.3)**2 / 0.1) + 0.02*np.random.randn(n_samples)
    geo_accuracy = np.clip(geo_accuracy, 0.7, 1.0)  # Clip to reasonable range
    
    # Combine outputs
    outputs = np.column_stack([residual_stress, porosity, geo_accuracy])
    
    return inputs, outputs, np.column_stack([x, y, z]), t.reshape(-1, 1)

# Generate data
inputs, outputs, coords, times = generate_synthetic_data(5000)

# Split into train/val/test
indices = np.random.permutation(len(inputs))
n_train = int(0.8 * len(indices))
n_val = int(0.1 * len(indices))

train_indices = indices[:n_train]
val_indices = indices[n_train:n_train+n_val]
test_indices = indices[n_train+n_val:]

train_inputs, train_outputs = inputs[train_indices], outputs[train_indices]
val_inputs, val_outputs = inputs[val_indices], outputs[val_indices]
test_inputs, test_outputs = inputs[test_indices], outputs[test_indices]

# Create a synthetic dataset file
data_file = Path('../data/processed/synthetic_dataset.h5')
data_file.parent.mkdir(parents=True, exist_ok=True)

with h5py.File(data_file, 'w') as f:
    # Training data
    train_group = f.create_group('train')
    train_group.create_dataset('inputs', data=train_inputs)
    train_group.create_dataset('outputs', data=train_outputs)
    train_group.create_dataset('coordinates', data=coords[train_indices])
    train_group.create_dataset('time', data=times[train_indices])
    
    # Extract scan vectors (just the process parameters)
    train_group.create_dataset('scan_vectors', data=train_inputs[:, :6])
    
    # Validation data
    val_group = f.create_group('val')
    val_group.create_dataset('inputs', data=val_inputs)
    val_group.create_dataset('outputs', data=val_outputs)
    val_group.create_dataset('coordinates', data=coords[val_indices])
    val_group.create_dataset('time', data=times[val_indices])
    val_group.create_dataset('scan_vectors', data=val_inputs[:, :6])
    
    # Test data
    test_group = f.create_group('test')
    test_group.create_dataset('inputs', data=test_inputs)
    test_group.create_dataset('outputs', data=test_outputs)
    test_group.create_dataset('coordinates', data=coords[test_indices])
    test_group.create_dataset('time', data=times[test_indices])
    test_group.create_dataset('scan_vectors', data=test_inputs[:, :6])
    
    # Metadata
    meta_group = f.create_group('metadata')
    meta_group.attrs['n_total'] = len(inputs)
    meta_group.attrs['n_train'] = len(train_indices)
    meta_group.attrs['n_val'] = len(val_indices)
    meta_group.attrs['n_test'] = len(test_indices)
    
    # Parameter names
    dt = h5py.special_dtype(vlen=str)
    param_names = np.array(['P', 'v', 'h', 'theta', 'l_island', 'layer_thickness'], dtype=object)
    meta_group.create_dataset('parameter_names', data=param_names, dtype=dt)

print(f"Created synthetic dataset at {data_file} with {len(inputs)} samples")
print(f"  Training: {len(train_indices)} samples")
print(f"  Validation: {len(val_indices)} samples")
print(f"  Test: {len(test_indices)} samples")

Let's visualize some of the relationships in our synthetic data:

In [ ]:
# Visualize relationship between process parameters and outputs
fig, axs = plt.subplots(3, 2, figsize=(16, 12))

# Plot residual stress vs. power and speed
axs[0, 0].scatter(inputs[:, 0], outputs[:, 0], alpha=0.3)
axs[0, 0].set_xlabel('Laser Power (W)')
axs[0, 0].set_ylabel('Residual Stress (MPa)')
axs[0, 0].set_title('Residual Stress vs. Laser Power')

axs[0, 1].scatter(inputs[:, 1], outputs[:, 0], alpha=0.3)
axs[0, 1].set_xlabel('Scan Speed (mm/s)')
axs[0, 1].set_ylabel('Residual Stress (MPa)')
axs[0, 1].set_title('Residual Stress vs. Scan Speed')

# Plot porosity vs. power and speed
axs[1, 0].scatter(inputs[:, 0], outputs[:, 1]*100, alpha=0.3)  # Convert back to percentage
axs[1, 0].set_xlabel('Laser Power (W)')
axs[1, 0].set_ylabel('Porosity (%)')
axs[1, 0].set_title('Porosity vs. Laser Power')

axs[1, 1].scatter(inputs[:, 1], outputs[:, 1]*100, alpha=0.3)  # Convert back to percentage
axs[1, 1].set_xlabel('Scan Speed (mm/s)')
axs[1, 1].set_ylabel('Porosity (%)')
axs[1, 1].set_title('Porosity vs. Scan Speed')

# Plot geometric accuracy vs. power and speed
axs[2, 0].scatter(inputs[:, 0], outputs[:, 2], alpha=0.3)
axs[2, 0].set_xlabel('Laser Power (W)')
axs[2, 0].set_ylabel('Geometric Accuracy')
axs[2, 0].set_title('Geometric Accuracy vs. Laser Power')

axs[2, 1].scatter(inputs[:, 1], outputs[:, 2], alpha=0.3)
axs[2, 1].set_xlabel('Scan Speed (mm/s)')
axs[2, 1].set_ylabel('Geometric Accuracy')
axs[2, 1].set_title('Geometric Accuracy vs. Scan Speed')

plt.tight_layout()
plt.show()

# Also show energy density plots (P/vh)
energy_density = inputs[:, 0] / (inputs[:, 1] * inputs[:, 2])

fig, axs = plt.subplots(1, 3, figsize=(18, 5))

axs[0].scatter(energy_density, outputs[:, 0], alpha=0.3)
axs[0].set_xlabel('Energy Density (J/mm³)')
axs[0].set_ylabel('Residual Stress (MPa)')
axs[0].set_title('Residual Stress vs. Energy Density')

axs[1].scatter(energy_density, outputs[:, 1]*100, alpha=0.3)  # Convert back to percentage
axs[1].set_xlabel('Energy Density (J/mm³)')
axs[1].set_ylabel('Porosity (%)')
axs[1].set_title('Porosity vs. Energy Density')

axs[2].scatter(energy_density, outputs[:, 2], alpha=0.3)
axs[2].set_xlabel('Energy Density (J/mm³)')
axs[2].set_ylabel('Geometric Accuracy')
axs[2].set_title('Geometric Accuracy vs. Energy Density')

plt.tight_layout()
plt.show()

## 3. PINN Model

Now let's create and train our Physics-Informed Neural Network (PINN) model.

In [ ]:
# Create a PINN model
model = PINN(
    in_dim=config['model']['input_dim'],
    out_dim=config['model']['output_dim'],
    width=config['model']['hidden_width'],
    depth=config['model']['hidden_depth']
)

print("PINN Model Architecture:")
print(model)

In [ ]:
# Configure the training process
# For the notebook demo, we'll use a simplified training setup
config['training']['n_epochs'] = 50  # Reduce epochs for the demo
config['training']['batch_size'] = 64
config['training']['print_freq'] = 5
config['data']['processed_data_path'] = '../data/processed/synthetic_dataset.h5'

# Initialize trainer
trainer = PINNTrainer(config_path)

# Load data for training
trainer.load_data()

# Train the model
print("Starting PINN training (this may take a while)...")
trainer.train()

# Plot training metrics
trainer.plot_metrics()

## 4. Model Evaluation

Let's evaluate our trained model on the test dataset.

In [ ]:
# Load the best model
best_model_path = list(trainer.checkpoint_dir.glob('best_model.pt'))[0]
trainer.load_checkpoint(best_model_path)

# Evaluate on test data
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test_data = torch.tensor(test_inputs, dtype=torch.float32).to(device)
test_labels = torch.tensor(test_outputs, dtype=torch.float32).to(device)

with torch.no_grad():
    model.eval()
    predictions = model(test_data)

# Convert to numpy for easier analysis
predictions = predictions.cpu().numpy()
test_labels = test_labels.cpu().numpy()

# Calculate metrics for each output
mse = np.mean((predictions - test_labels)**2, axis=0)
mae = np.mean(np.abs(predictions - test_labels), axis=0)
r2 = 1 - np.sum((predictions - test_labels)**2, axis=0) / np.sum((test_labels - np.mean(test_labels, axis=0))**2, axis=0)

print("Test set evaluation:")
output_names = ['Residual Stress', 'Porosity', 'Geometric Accuracy']
for i, name in enumerate(output_names):
    print(f"{name}:")
    print(f"  MSE: {mse[i]:.4f}")
    print(f"  MAE: {mae[i]:.4f}")
    print(f"  R²: {r2[i]:.4f}")

# Plot predicted vs. actual for each output
fig, axs = plt.subplots(1, 3, figsize=(18, 5))

for i, (name, ax) in enumerate(zip(output_names, axs)):
    ax.scatter(test_labels[:, i], predictions[:, i], alpha=0.3)
    
    # Add diagonal line (perfect prediction)
    min_val = min(np.min(test_labels[:, i]), np.min(predictions[:, i]))
    max_val = max(np.max(test_labels[:, i]), np.max(predictions[:, i]))
    ax.plot([min_val, max_val], [min_val, max_val], 'r--')
    
    ax.set_xlabel(f'Actual {name}')
    ax.set_ylabel(f'Predicted {name}')
    ax.set_title(f'{name}: R² = {r2[i]:.4f}')

plt.tight_layout()
plt.show()

## 5. Multi-Objective Optimization

Now that we have a trained surrogate model, let's use it for multi-objective optimization using NSGA-III.

In [ ]:
# Initialize the optimizer
optimizer = NSGAOptimizer(config_path, best_model_path)

# Run the optimization
print("Running NSGA-III optimization (this may take a while)...")
results = optimizer.optimize()

# Extract Pareto front solutions
pareto_params = results.X  # Process parameters on the Pareto front
pareto_objectives = results.F  # Corresponding objective values

print(f"Optimization complete with {len(pareto_params)} solutions on the Pareto front")

# Plot the Pareto front
if pareto_objectives.shape[1] == 2:
    # 2D Pareto front
    plt.figure(figsize=(10, 8))
    plt.scatter(pareto_objectives[:, 0], pareto_objectives[:, 1], s=30)
    plt.xlabel(config['optimizer']['objectives'][0])
    plt.ylabel(config['optimizer']['objectives'][1])
    plt.title('Pareto Front')
    plt.grid(True)
    plt.show()
    
elif pareto_objectives.shape[1] == 3:
    # 3D Pareto front
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(pareto_objectives[:, 0], pareto_objectives[:, 1], pareto_objectives[:, 2], s=30)
    ax.set_xlabel(config['optimizer']['objectives'][0])
    ax.set_ylabel(config['optimizer']['objectives'][1])
    ax.set_zlabel(config['optimizer']['objectives'][2])
    ax.set_title('Pareto Front')
    plt.show()

# Display some of the optimal solutions
param_names = list(config['optimizer']['param_bounds'].keys())
objective_names = config['optimizer']['objectives']

print("\nSample optimal solutions from the Pareto front:")
print("-" * 80)
print(f"{'Solution':^10} | " + " | ".join([f"{name:^12}" for name in param_names]) + " | " + 
      " | ".join([f"{name:^18}" for name in objective_names]))
print("-" * 80)

# Sample solutions (every 10th solution or fewer if there are fewer than 50 solutions)
step = max(1, len(pareto_params) // 5)
for i in range(0, len(pareto_params), step):
    param_str = " | ".join([f"{val:12.4f}" for val in pareto_params[i]])
    obj_str = " | ".join([f"{val:18.4f}" for val in pareto_objectives[i]])
    print(f"{i:^10} | {param_str} | {obj_str}")

print("-" * 80)

In [ ]:
# Explore the trade-offs between objectives
from sklearn.decomposition import PCA

# Apply PCA to process parameters to find primary directions of variation
pca = PCA(n_components=2)
params_pca = pca.fit_transform(pareto_params)

# Create scatter plot colored by each objective
fig, axs = plt.subplots(1, 3, figsize=(18, 5))

for i, (name, ax) in enumerate(zip(objective_names, axs)):
    scatter = ax.scatter(params_pca[:, 0], params_pca[:, 1], 
                         c=pareto_objectives[:, i], cmap='viridis', 
                         s=50, alpha=0.8)
    ax.set_xlabel('PCA Component 1')
    ax.set_ylabel('PCA Component 2')
    ax.set_title(f'Parameter Space Colored by {name}')
    fig.colorbar(scatter, ax=ax, label=name)

plt.tight_layout()
plt.show()

# Display PCA component weights
print("PCA Component Weights (Contribution of each parameter to the principal components):")
for i, comp in enumerate(pca.components_):
    print(f"Component {i+1}:")
    for name, weight in zip(param_names, comp):
        print(f"  {name}: {weight:.4f}")

## 6. Selecting Optimal Process Parameters

Based on the Pareto front, let's select a few process parameter sets for validation.

In [ ]:
# Select solutions with desired properties
# For example, select solutions with minimal residual stress
min_stress_idx = np.argmin(pareto_objectives[:, 0])
min_stress_params = pareto_params[min_stress_idx]
min_stress_objectives = pareto_objectives[min_stress_idx]

# Minimal porosity
min_porosity_idx = np.argmin(pareto_objectives[:, 1])
min_porosity_params = pareto_params[min_porosity_idx]
min_porosity_objectives = pareto_objectives[min_porosity_idx]

# Maximum geometric accuracy
# Assuming here that objective is negative geometric accuracy (to be minimized)
max_geo_idx = np.argmin(pareto_objectives[:, 2])  
max_geo_params = pareto_params[max_geo_idx]
max_geo_objectives = pareto_objectives[max_geo_idx]

# Balanced solution (middle of Pareto front)
# One approach: find solution with minimum Euclidean distance to the utopia point
# First normalize objectives to [0,1] scale
obj_min = np.min(pareto_objectives, axis=0)
obj_max = np.max(pareto_objectives, axis=0)
obj_range = obj_max - obj_min
normalized_objectives = (pareto_objectives - obj_min) / obj_range

# Utopia point is [0,0,0] in normalized space
distances = np.sqrt(np.sum(normalized_objectives**2, axis=1))
balanced_idx = np.argmin(distances)
balanced_params = pareto_params[balanced_idx]
balanced_objectives = pareto_objectives[balanced_idx]

# Display the selected solutions
print("\nSelected Solutions for Validation:")
print("-" * 80)
print(f"{'Criterion':^20} | " + " | ".join([f"{name:^12}" for name in param_names]) + " | " + 
      " | ".join([f"{name:^18}" for name in objective_names]))
print("-" * 80)

solutions = [
    ("Min Residual Stress", min_stress_params, min_stress_objectives),
    ("Min Porosity", min_porosity_params, min_porosity_objectives),
    ("Max Geo Accuracy", max_geo_params, max_geo_objectives),
    ("Balanced", balanced_params, balanced_objectives)
]

for name, params, objectives in solutions:
    param_str = " | ".join([f"{val:12.4f}" for val in params])
    obj_str = " | ".join([f"{val:18.4f}" for val in objectives])
    print(f"{name:^20} | {param_str} | {obj_str}")

print("-" * 80)

# Save selected parameters for validation
validation_file = Path('../data/optimized/validation_parameters.h5')
validation_file.parent.mkdir(parents=True, exist_ok=True)

with h5py.File(validation_file, 'w') as f:
    # Save parameters
    all_params = np.vstack([params for _, params, _ in solutions])
    f.create_dataset('parameters', data=all_params)
    
    # Save objectives
    all_objectives = np.vstack([obj for _, _, obj in solutions])
    f.create_dataset('objectives', data=all_objectives)
    
    # Save parameter and objective names
    dt = h5py.special_dtype(vlen=str)
    param_names_array = np.array(param_names, dtype=object)
    obj_names_array = np.array(objective_names, dtype=object)
    solution_names = np.array([name for name, _, _ in solutions], dtype=object)
    
    f.create_dataset('parameter_names', data=param_names_array, dtype=dt)
    f.create_dataset('objective_names', data=obj_names_array, dtype=dt)
    f.create_dataset('solution_names', data=solution_names, dtype=dt)

print(f"Saved validation parameters to {validation_file}")

## 7. Summary

In this notebook, we've demonstrated the complete workflow for physics-informed optimization of LPBF process parameters:

1. Generated synthetic data that mimics FEA simulation results
2. Trained a physics-informed neural network (PINN) as a surrogate model
3. Validated the model on test data
4. Used NSGA-III multi-objective optimization to find the Pareto front
5. Selected optimal parameter sets for different objectives

The next steps would be to:

1. Build physical coupons using the optimized parameters
2. Characterize the resulting parts (porosity, residual stress, geometric accuracy)
3. Compare the experimental results to model predictions
4. Refine the model with the new experimental data

This completes the feedback loop, allowing for continuous improvement of the surrogate model and optimization process.